In [1]:
import pandas as pd


In [10]:
df = pd.read_csv('C:\\Users\\ahmed\\OneDrive\\Documents\\GitHub\\Arabic-Sentiment-Analysis-using-Arabic-BERT\\twitter_data.csv')


In [11]:
df.head()


,label,tweet
0,0,اعترف ان بتس كانو شوي شوي يجيبو راسي لكن اليوم...
1,0,توقعت اذا جات داريا بشوفهم كاملين بس لي للحين ...
2,0,#الاهلي_الهلال اكتب توقعك لنتيجة لقاء الهلال و...
3,0,نعمة المضادات الحيوية . تضع قطرة💧مضاد بنسلين ع...
4,0,الدودو جايه تكمل علي 💔


In [12]:
!pip install pymongo

  Using cached dnspython-2.6.1-py3-none-any.whl.metadata (5.8 kB)
   ---------------------------------------- 0.0/582.0 kB ? eta -:--:--
   ---------------------------------------- 582.0/582.0 kB 5.1 MB/s eta 0:00:00
Using cached dnspython-2.6.1-py3-none-any.whl (307 kB)


In [13]:
import pandas as pd
from pymongo import MongoClient

# Load the DataFrame (assuming you've already done this)
df = pd.read_csv(r'C:\Users\ahmed\OneDrive\Documents\GitHub\Arabic-Sentiment-Analysis-using-Arabic-BERT\twitter_data.csv')

# Convert the DataFrame to a dictionary format suitable for MongoDB
data_dict = df.to_dict("records")

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')  # Adjust the URI if necessary

# Specify the database and collection
db = client['twitter']  # Replace 'my_database' with your database name
collection = db['total_tweets']

# Insert the data into the collection
collection.insert_many(data_dict)

print("Data successfully inserted into MongoDB collection 'total_tweets'")


Data successfully inserted into MongoDB collection 'total_tweets'


In [4]:
os.chdir("c:\\Users\\ahmed\\OneDrive\\Documents\\GitHub\\Arabic-Sentiment-Analysis-using-AraGPT2\\data")


In [6]:
%pwd

'c:\\Users\\ahmed\\OneDrive\\Documents\\GitHub\\Arabic-Sentiment-Analysis-using-AraGPT2\\data'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
  root_dir: Path
  twitter_data: Path
  Arabic_SS2030: Path
  ar_reviews_100k: Path


In [ ]:
from Sentiment.constant import *
from Sentiment.utils.common import read_yaml, create_directories


In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [ ]:
import urllib.request as request
import zipfile
from Sentiment.logger import logger
from Sentiment.utils.common import get_size

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

In [1]:
MODEL_NAME :str = 'openai-community/roberta-base-openai-detector'

In [6]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, RandomSampler, TensorDataset
from sklearn.metrics import accuracy_score
from transformers import AutoModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from Sentiment.entity.artifact_entity import ModelTrainerArtifact, DataTransformationArtifact
from Sentiment.entity.config_entity import ModelTrainerConfig
from Sentiment.exception import SentimentException
from Sentiment.utils.main_utils import save_object, load_object
from Sentiment.constant.training_pipeline import MODEL_NAME
import numpy as np
from Sentiment.logger import logging
from numpy import ndarray
import sys

import torch
import torch.nn as nn
from transformers import AutoModel

import torch
import torch.nn as nn
from transformers import AutoModel
import concurrent.futures


In [8]:
class RobertaClassifier(nn.Module):
    def __init__(self, tokenizer=None, freeze_roberta=False):
        super(RobertaClassifier, self).__init__()

        D_in = 768  # Input dimension based on the roberta-base model
        H, D_out = 50, 2  # Intermediate layer size and output dimension
        self.tokenizer = tokenizer
        self.roberta = AutoModel.from_pretrained(MODEL_NAME)
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        if freeze_roberta:
            for param in self.roberta.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        # Debugging: Check input tensor shapes
        print(f"Forward pass input_ids shape: {input_ids.shape}")
        print(f"Forward pass attention_mask shape: {attention_mask.shape}")

        if input_ids.size(0) == 0:
            raise ValueError("Empty input_ids tensor")
        if attention_mask.size(0) == 0:
            raise ValueError("Empty attention_mask tensor")

        # Ensure the sequence length is within the model's maximum sequence length
        if input_ids.size(1) > self.roberta.config.max_position_embeddings:
            raise ValueError(f"Sequence length {input_ids.size(1)} exceeds maximum allowed length "
                             f"{self.roberta.config.max_position_embeddings}")

        # Feed input to RoBERTa
        try:
            outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        except Exception as e:
            print(f"Error during RoBERTa forward pass: {e}")
            raise

        # Extract the last hidden state of the token [CLS] for classification task
        try:
            last_hidden_state_cls = outputs[0][:, 0, :]
            print(f"Last hidden state [CLS] shape: {last_hidden_state_cls.shape}")
        except IndexError as e:
            print(f"Index error in extracting [CLS] token: {e}")
            raise

        # Feed input to classifier to compute logits
        try:
            logits = self.classifier(last_hidden_state_cls)
            print(f"Logits shape: {logits.shape}")
        except Exception as e:
            print(f"Error during classifier forward pass: {e}")
            raise


In [12]:
from transformers import AutoTokenizer

def preprocess_data(texts, tokenizer, max_length=128):
    """
    Preprocesses the input texts by tokenizing and converting to tensors.
    
    Args:
        texts (list): List of input strings to be tokenized.
        tokenizer (AutoTokenizer): The tokenizer instance from Hugging Face.
        max_length (int): The maximum sequence length for the tokens.

    Returns:
        input_ids (torch.Tensor): Tensor containing token ids.
        attention_mask (torch.Tensor): Tensor containing attention masks.
    """
    # Tokenize the input texts
    encoding = tokenizer(
        texts,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']
    
    return input_ids, attention_mask


In [14]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Example texts to classify
texts = ["Example text for classification", "Another example text"]

# Preprocess the texts
input_ids, attention_mask = preprocess_data(texts, tokenizer, max_length=141)

# Forward pass through the model
logits = model(input_ids, attention_mask)


Forward pass input_ids shape: torch.Size([2, 141])
Forward pass attention_mask shape: torch.Size([2, 141])
Last hidden state [CLS] shape: torch.Size([2, 768])
Logits shape: torch.Size([2, 2])


In [28]:
import pandas as pd
from pymongo import MongoClient
import sys

class TSVtoMongoDB:
    def __init__(self, tsv_file_path, mongo_uri, database_name, collection_name):
        self.tsv_file_path = tsv_file_path
        self.mongo_uri = mongo_uri
        self.database_name = database_name
        self.collection_name = collection_name

    def read_tsv(self):
        try:
            # Read the TSV file into a pandas DataFrame
            df = pd.read_csv(self.tsv_file_path, sep='\t')
            return df
        except Exception as e:
            print(f"Error reading the TSV file: {e}")
            sys.exit(1)

    def convert_labels(self, df):
        """
        Convert 'label' column from 'Positive'/'Negative'/'Mixed' to 1/0/2.
        """
        try:
            if 'label' not in df.columns:
                raise ValueError("The DataFrame does not contain a 'label' column.")
            
            # Map the labels to 1, 0, and 2
            df['label'] = df['label'].map({'Positive': 1, 'Negative': 0, 'Mixed': 2})
            
            if df['label'].isnull().any():
                print(df['label'].unique())
                raise ValueError("There are values in the 'label' column that are not mapped to 1, 0, or 2.")
            
            print("Labels have been converted to 0, 1, and 2.")
        except Exception as e:
            print(f"Error converting labels: {e}")
            sys.exit(1)

    def save_to_mongo(self, df):
        try:
            # Establish connection to MongoDB
            client = MongoClient(self.mongo_uri)
            db = client[self.database_name]
            collection = db[self.collection_name]

            # Convert DataFrame to a list of dictionaries and insert into MongoDB
            data = df.to_dict(orient='records')
            collection.insert_many(data)
            print(f"Inserted {len(data)} records into {self.collection_name} collection in MongoDB.")
        except Exception as e:
            print(f"Error saving data to MongoDB: {e}")
            sys.exit(1)

    def process(self):
        try:
            df = self.read_tsv()
            self.convert_labels(df)  # Convert labels before saving to MongoDB
            self.save_to_mongo(df)
        except Exception as e:
            print(f"An error occurred during processing: {e}")
            sys.exit(1)

# Usage example:
if __name__ == "__main__":
    tsv_file_path = r"C:\Users\ahmed\Downloads\ar_reviews_100k.tsv"
    mongo_uri = "mongodb://localhost:27017/"  # Replace with your MongoDB URI
    database_name = "ar_reviews"
    collection_name = "reviews"

    tsv_to_mongo = TSVtoMongoDB(tsv_file_path, mongo_uri, database_name, collection_name)
    tsv_to_mongo.process()


Labels have been converted to 0, 1, and 2.
Inserted 99999 records into reviews collection in MongoDB.


In [29]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))


False
0


AssertionError: Torch not compiled with CUDA enabled

In [24]:
df['label'].unique()

KeyError: 'label'

In [17]:
from pymongo import MongoClient

# Replace with your actual connection string and collection details
client = MongoClient('mongodb://localhost:27017')
db = client['restaurant']
collection = db['reviews']

# Fetch one document to verify the connection
doc = collection.find_one()
if doc:
    print("Successfully connected to MongoDB and fetched a document.")
else:
    print("Connection successful, but no documents found in the collection.")


Successfully connected to MongoDB and fetched a document.


In [18]:
import pandas as pd

# Fetch all documents from the collection
data = list(collection.find())

# Convert to DataFrame
if data:
    df = pd.DataFrame(data)
    print(f"Data successfully loaded into DataFrame. Shape: {df.shape}")
else:
    print("No data found in the collection.")


Data successfully loaded into DataFrame. Shape: (1000, 3)


In [19]:
df.head()

,_id,Review,Liked
0,66c2d092cb5e6a0889a6529e,Wow... Loved this place.,1
1,66c2d092cb5e6a0889a6529f,Crust is not good.,0
2,66c2d092cb5e6a0889a652a0,Not tasty and the texture was just nasty.,0
3,66c2d092cb5e6a0889a652a1,Stopped by during the late May bank holiday of...,1
4,66c2d092cb5e6a0889a652a2,The selection on the menu was great and so wer...,1
